#### This file is used for the testing of the data processing for a few articles instead of the whole parquet file

#### Instructions before running all the cells:
1. Change the input directory to the path that you stored the "merged_data_rag.parquet" and the respective output directory to your intended output folders
2. Install the required packages below

In [1]:
!pip install azure-cli pandas openai fastparquet pyarrow


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import glob
import json
import os
from tqdm import tqdm

import pandas as pd
from azure.identity import AzureCliCredential, get_bearer_token_provider
from openai import AzureOpenAI

#### Pre-processing the merged parquet file

In [ ]:
df = pd.read_parquet("merged_data.parquet")

In [ ]:
df[df["remove_type"] == "excel_error"].shape[0]

In [ ]:
df.loc[df["remove_type"] == "No HTML Tags"].shape[0]

In [ ]:
# Remove articles with 'No HTML Tags' from the 'remove_type' column
df = df.loc[df["remove_type"] != "No HTML Tags"]

In [ ]:
df.loc[df["remove_type"] == "No HTML Tags"].shape[0]

In [ ]:
df.loc[df["remove_type"] == "No Extracted Content"].shape[0]

In [ ]:
# Remove the rows with 'No Extracted Content' from 'remove_type' column
df = df[df["remove_type"] != "No Extracted Content"]

In [ ]:
df.loc[df["remove_type"] == "No Extracted Content"].shape[0]

In [ ]:
df.loc[df["remove_type"] == "NaN"].shape[0]

In [ ]:
# Remove the rows with 'NaN' from 'remove_type' column
df = df[df["remove_type"] != "NaN"]

In [ ]:
df.loc[df["remove_type"] == "NaN"].shape[0]

In [ ]:
df.loc[df["remove_type"] == "Multilingual"].shape[0]

In [ ]:
# Remove 'Multilingual' from 'remove_type' column
df = df[df["remove_type"] != "Multilingual"]

In [ ]:
df.loc[df["remove_type"] == "Multilingual"].shape[0]

In [ ]:
df.shape[0]

In [ ]:
df.loc[df["id"].isin([1444496, 1445629, 1443608, 1435183, 1435335, 1434652])]

In [ ]:
# Remove the duplicated articles with the 'id' = "1445629", "1443608", "1435183", "1435335", "1434652".
df = df[~df["id"].isin([1444496, 1445629, 1443608, 1435183, 1435335, 1434652])]

In [ ]:
df.loc[df["id"].isin([1444496, 1445629, 1443608, 1435183, 1435335, 1434652])]

In [ ]:
df.loc[df["id"].isin([1497409, 1469472])]

In [ ]:
df.loc[df["remove_type"] == "Duplicated Content"].shape[0]

In [ ]:
df.shape[0]

In [ ]:
# Remove all the 'Duplicated Content' from 'remove_type' column and only keep the articles with the 'id' = "1497409", "1469472".
df = df[(df["remove_type"] != "Duplicated Content") | (df["id"].isin([1497409, 1469472]))]

In [ ]:
df.shape[0]

In [ ]:
# Save the cleaned data to a new parquet file with the name 'merged_data_rag.parquet'
df.to_parquet("merged_data_rag.parquet")

In [ ]:
# Count the data in the 'content_category' column
df["content_category"].value_counts()

#### Creating a new parquet file with selected articles for testing

In [80]:
# Load the original Parquet file
input_file = "/Users/Richmond/Desktop/syn/healthhub-content-optimization/content-optimization/data/03_primary/merged_data.parquet/merged_data_rag.parquet"
df = pd.read_parquet(input_file)

# Filter the DataFrame to get the rows with specific article IDs
article_id_to_extract = [1445517]
filtered_df = df[df["id"].isin(article_id_to_extract)]

# Ensure the data_processing directory exists
output_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test"
os.makedirs(output_dir, exist_ok=True)

# Save the filtered DataFrame as a new Parquet file
output_file = os.path.join(output_dir, "index.parquet")
filtered_df.to_parquet(output_file, index=False)

print(f"Extracted articles {article_id_to_extract} and saved to {output_file}")

Extracted articles [1445517] and saved to /Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/index.parquet


#### Read the pre-processed parquet file

In [83]:
df = pd.read_parquet("/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/index.parquet")

In [84]:
df

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,extracted_content_body,l1_mappings,l2_mappings,page_views,engagement_rate,bounce_rate,exit_rate,scroll_percentage,percentage_total_views,cumulative_percentage_total_views
0,1445517,Healthy Ways to Manage Your Weight,Healthy Ways to Manage Your Weight,"Exercise and Fitness,Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/Manage W...,www.healthhub.sg/live-healthy/Manage Weight He...,Manage Weight Healthily,Finding it a struggle to manage your weight? U...,"b'<div class=""ExternalClassA1CC8F76D57D4D87888...",...,Weight management - it's easier than you think...,Well-being & Lifestyle,"Exercise and Fitness | Food, Diet and Nutrition",1455,0.972685,0.027315,0.107216,0.271478,0.000625,0.789989


#### Pass the table content into GPT-4o and create a new parquet with the new column "processed_table_content"

In [85]:
def ask(html_content: str) -> str:
    azure_credential = AzureCliCredential()
    token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")

    openai_client = AzureOpenAI(
        api_version="2024-06-01",
        azure_endpoint="https://apim-jisfkas7teqvm.azure-api.net",
        azure_ad_token_provider=token_provider,
    )

    # Updated prompt to discourage repetition
    prompt = """
    Below is the given full article HTML. Extract the **content of the tables** and their **relevant descriptions** that help understand the tables. 
    Ensure:
    - Retain only essential markdown formatting, such as:
        - **Bold** for headers or important table titles.
        - **Tables** formatted using markdown syntax (e.g., `| Header 1 | Header 2 |`).
    - Avoid unnecessary dashes, bullet points, and extraneous markdown symbols.
    - Remove all other HTML tags.
    - Keep the output concise, accurate, and under 4,000 words. If it exceeds 4,000 words, prioritize summarization.
    - Output the response as a readable markdown string.

    {html_content}
    """

    # Prepare the messages for the API call
    query_messages = [
        {
            "role": "system",
            "content": "You are an AI assistant specialized in extracting structured content from HTML.",
        },
        {"role": "user", "content": prompt.format(html_content=html_content)},
    ]

    response = openai_client.chat.completions.create(
        messages=query_messages,
        model="chat",
        temperature=0.0,
        max_tokens=4096,
        n=1,
        seed=1234,
    )

    return response.choices[0].message.content


# Extract and process HTML tables


def process_html_tables(row):
    if row["has_table"]:
        return ask(row["content_body"])
    return None


# Apply processing to the DataFrame with tqdm for progress tracking
tqdm.pandas()  # Enable progress bar for DataFrame operations
df_filtered["processed_table_content"] = df_filtered.progress_apply(process_html_tables, axis=1)

# Save to a new Parquet file
output_file = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/new_index.parquet"
df.to_parquet(output_file)

#### Read the post-processed parquet file

In [89]:
df1 = pd.read_parquet("/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/new_index.parquet")

In [94]:
df1 = pd.read_parquet("/Users/Richmond/Desktop/syn/azure-conversational-assistant/data_processing/new_index.parquet")

#### Find the top k table length after processing

In [96]:
# Ensure 'processed_table_content' column does not contain None values
df1_filtered = df1[df1["processed_table_content"].notnull() & df1["has_table"]]

# Find the lengths of each string in the "processed_table_content" column
df1_filtered["content_length"] = df1_filtered["processed_table_content"].apply(len)

# Sort the dataframe by content length in descending order and get the top 17
top_30_longest = df1_filtered.nlargest(17, "content_length")

# Display the relevant columns for the top 17 entries
top_30_longest[["processed_table_content", "content_length", "id", "friendly_url", "content_category"]]

/var/folders/xd/bj3151rn7cv92frd9tzmsg4r0000gp/T/ipykernel_32558/669495784.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['content_length'] = df1_filtered['processed_table_content'].apply(len)


,processed_table_content,content_length,id,friendly_url,content_category
378,**Recommended Dietary Allowances for Normal He...,13092,1444820,recommended_dietary_allowances,live-healthy-articles
950,**Food and Water Borne Diseases**\n\nHere are ...,12028,1445538,travellersurvivalguide,live-healthy-articles
369,**How Much Protein do you Need?**\n\n| Age Ran...,10556,1445282,seniors-need-more-protein,live-healthy-articles
2525,**Symptoms and Reasons for Feeling Worse After...,10274,1434716,IQuit,programs
1705,**A. Insulin vial (small cylindrical glass con...,9849,1440453,Insulin-Injection-Technique,medications
773,**Eat Healthily and Stay Active**\n\nHelping y...,9837,1442929,growing-kid-raising-healthy-kids,live-healthy-articles
357,# Admissions and Outpatient Attendances\n\n**H...,9548,1437940,admissions-and-outpatient-attendances,health-statistics
553,**Nutritional Requirements for Toddlers after ...,9538,1445172,nutrition-for-toddlers-25-36-months-old,live-healthy-articles
1545,**About Respite Care for Caregivers**\n\nCareg...,8562,1437970,respite_care,medical-care-and-facilities
1519,**Meeting nutritional needs of kids and teenag...,8396,1445673,A Healthy Food Foundation - for Kids and Teens,live-healthy-articles


In [95]:
# Filter the dataframe to find the row with id 1440254
content = df1[df1["id"] == 1445517]["processed_table_content"].values[0]

# Write the content to a text file
with open("processed_table_content_1440254.txt", "w") as file:
    file.write(content)

print("Content has been written to processed_table_content_1440254.txt")

Content has been written to processed_table_content_1440254.txt


#### Inspect the top k articles post-processed "processed_table_content" 

In [ ]:
# # Step 2: Filter the DataFrame by the specific IDs
# filtered_df = df1[df1['id'].isin([1439448, 1445517])]

# # Step 3: Extract the 'processed_table_content' column for the selected rows
# content_1439448 = filtered_df[filtered_df['id'] == 1439448]['processed_table_content'].values[0]
# content_1445517 = filtered_df[filtered_df['id'] == 1445517]['processed_table_content'].values[0]

# # Step 4: Write the content to a .txt file
# with open("processed_content_1439448.txt", "w") as file1:
#     file1.write(content_1439448)

# with open("processed_content_1445517.txt", "w") as file2:
#     file2.write(content_1445517)

# print("Files saved successfully.")

#### Manually curate the content to ensure the quality of the data for ingestion into the search index

In [ ]:
# # Step 2: Define the new content for each ID
# new_content_1439448 = """
# **Week 3 of the pack:**

# | No missed tablets in the last 7 days | Take the missed tablet as soon as remembered, even if it means taking 2 tablets at the same time. Continue to take your tablets at your usual time and start the next pack right away without the 7-day tablet free period or 7-day white (inactive) tablets i.e. no gap should be left between packs. Your menses may not come until the next pack is finished, but there is no need to worry. However, if your menses do not occur after the next pack is finished, you should take a pregnancy test to make sure you are not pregnant. OR Stop taking medication from the current pack for 7 days (7-day tablet-free period). A withdrawal bleed (menses) usually occurs and then start a next pack after 7 days. |
# """

# new_content_1445517 = """
# **Suggestions for Overcoming Physical Activity Barriers**

# | Lack of time | Monitor your daily activities for one week. Identify available time slots where you can get at least 10 minutes of aerobic type physical activity. Add physical activity to your daily routine. Walk or ride your bicycle to work or to the shops, and organise your daily activities around physical activity. E.g. walk the dog, exercise while you watch TV, park farther away from your destination. Select activities requiring minimal time, such as walking, jogging or stair climbing. |
# """

# # Step 3: Update the 'processed_table_content' for the given IDs
# df.loc[df1['id'] == 1439448, 'processed_table_content'] = new_content_1439448
# df.loc[df1['id'] == 1445517, 'processed_table_content'] = new_content_1445517

# # Step 4: Save the updated DataFrame to a new Parquet file
# df1.to_parquet("data_processing/new_index.parquet")

# print("Updated content and saved to Parquet file successfully.")

#### Extract out the "content_body" as a .txt file to verify extraction

In [62]:
# Filter the DataFrame for rows where 'has_table' is True
filtered_df1 = df1[df1["has_table"]]

# Ensure the raw_content_body directory exists
raw_content_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/html_tables_cleaning"
os.makedirs(raw_content_dir, exist_ok=True)

# Process each article ID
for article_id in filtered_df1["id"].unique():
    # Filter the DataFrame for the current article ID
    article_df = filtered_df1[filtered_df1["id"] == article_id]

    # Extract the 'content_body' column
    processed_content = article_df["content_body"].tolist()

    # Define the output file path
    output_txt_file = f"{raw_content_dir}/raw_article_{article_id}.txt"

    # Write the content to a text file
    with open(output_txt_file, "w") as file:
        for content in processed_content:
            # Check if content is in bytes and decode if necessary
            if isinstance(content, bytes):
                content = content.decode("utf-8")
            file.write(content + "\n")  # Write each entry on a new line

    print(f"Raw content body for article {article_id} saved to {output_txt_file}")

Raw content body for article 1435005 saved to ./data_processing/raw_content_body/raw_article_1435005.txt
Raw content body for article 1434994 saved to ./data_processing/raw_content_body/raw_article_1434994.txt
Raw content body for article 1435059 saved to ./data_processing/raw_content_body/raw_article_1435059.txt
Raw content body for article 1434998 saved to ./data_processing/raw_content_body/raw_article_1434998.txt
Raw content body for article 1464135 saved to ./data_processing/raw_content_body/raw_article_1464135.txt
Raw content body for article 1445643 saved to ./data_processing/raw_content_body/raw_article_1445643.txt


#### Extract out the "processed_table_content" as a .txt file to verify extraction

In [69]:
# Filter the DataFrame for rows where 'has_table' is True
filtered_df = df1[df1["has_table"]]

# Ensure the raw_content_body directory exists
processed_content_dir = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/html_tables_cleaning"
os.makedirs(processed_content_dir, exist_ok=True)

# Loop through each article ID to process
for article_id in article_id_to_extract:
    # Filter the DataFrame for the specific article ID
    article_df = filtered_df[filtered_df["id"] == article_id]

    if not article_df.empty:
        # Extract the 'processed_table_content' column
        processed_content = article_df["processed_table_content"].astype(str).tolist()

        # Define the output file path
        output_txt_file = f"{processed_content_dir}/processed_table_content_{article_id}.txt"

        # Write the content to a text file
        with open(output_txt_file, "w") as file:
            for content in processed_content:
                file.write(content + "\n")  # Write each entry on a new line

        print(f"Processed table content for article {article_id} saved to {output_txt_file}")

Processed table content for article 1464135 saved to ./data_processing/processed_table_content/processed_table_content_1464135.txt
Processed table content for article 1445643 saved to ./data_processing/processed_table_content/processed_table_content_1445643.txt
Processed table content for article 1434994 saved to ./data_processing/processed_table_content/processed_table_content_1434994.txt
Processed table content for article 1435005 saved to ./data_processing/processed_table_content/processed_table_content_1435005.txt
Processed table content for article 1434998 saved to ./data_processing/processed_table_content/processed_table_content_1434998.txt
Processed table content for article 1435059 saved to ./data_processing/processed_table_content/processed_table_content_1435059.txt


#### Check that processed_table_content is added as the last column of the new index

In [70]:
# Print all columns of the DataFrame
print("Columns in DataFrame:")
for column in df1.columns:
    print(column)

Columns in DataFrame:
id
content_name
title
article_category_names
cover_image_url
full_url
full_url2
friendly_url
category_description
content_body
keywords
feature_title
pr_name
alternate_image_text
date_modified
number_of_views
last_month_view_count
last_two_months_view
page_views
engagement_rate
bounce_rate
exit_rate
scroll_percentage
percentage_total_views
cumulative_percentage_total_views
content_category
to_remove
remove_type
has_table
has_image
related_sections
extracted_tables
extracted_raw_html_tables
extracted_links
extracted_headers
extracted_images
extracted_content_body
l1_mappings
l2_mappings
processed_table_content


#### Specify the columns to extract for article content and tables

In [71]:
columns_to_extract1 = [
    "id",
    "title",
    "cover_image_url",
    "full_url",
    "extracted_content_body",
    "content_category",
    "category_description",
    "has_table",  # Add this column to filter rows with tables after extracting the content
]

columns_to_extract2 = [
    "id",
    "title",
    "cover_image_url",
    "full_url",
    "processed_table_content",
    "content_category",
    "category_description",
]

#### Check the article count with tables and output their "full_url"

In [72]:
# List of article IDs to check
article_ids = [
    1442739,
    1445332,
    1439757,
    1439210,
    1443217,
    1435054,
    1437631,
    1434620,
    1444553,
    1445932,
    1437772,
    1435185,
    1435017,
    1464135,
    1445643,
    1434994,
    1435005,
    1434998,
    1435059,
]

# Filter the DataFrame for the rows with the specified article IDs
filtered_df = df[df["id"].isin(article_ids)]

# Check which rows have 'has_table' == True
articles_with_tables = filtered_df[filtered_df["has_table"]]

# Count of articles with tables
has_table_count = articles_with_tables.shape[0]

# Extract article IDs and full URLs
article_info = articles_with_tables[["id", "full_url"]]

# Adjust display settings for full URL visibility
pd.set_option("display.max_colwidth", None)

# Print the count and article info
print(f"Number of articles with 'has_table' == True: {has_table_count}")
print("Articles with tables:")
print(article_info)

# Optionally reset display settings if needed
pd.reset_option("display.max_colwidth")

Number of articles with 'has_table' == True: 6
Articles with tables:
        id  \
0  1435005   
1  1434994   
2  1435059   
3  1434998   
6  1464135   
7  1445643   

                                                                                 full_url  
0             https://www.healthhub.sg/a-z/costs-and-financing/enhancement_active_seniors  
1  https://www.healthhub.sg/a-z/costs-and-financing/subsidies_intermediate_long_term_care  
2                            https://www.healthhub.sg/a-z/costs-and-financing/eldershield  
3                https://www.healthhub.sg/a-z/costs-and-financing/assistance_scheme_IDAPE  
6           https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work  
7                                https://www.healthhub.sg/live-healthy/gettingsupportstis  


#### Specific articles extraction

In [76]:
# List of article IDs to extract
article_ids = [
    1442739,
    1445332,
    1439757,
    1439210,
    1443217,
    1435054,
    1437631,
    1434620,
    1444553,
    1445932,
    1437772,
    1435185,
    1435017,
    1464135,
    1445643,
    1434994,
    1435005,
    1434998,
    1435059,
]

# Specify the directory
output_directory = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/processed_articles"

# Ensure the directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop through each article ID in the list
for article_id in article_ids:
    # Extract the row based on the article ID
    row = df1[df1["id"] == article_id]

    # Check if row is empty
    if row.empty:
        print(f"Article ID {article_id} not found in the DataFrame.")
        continue

    row_number = row.index[0]  # Get the index of the row

    # Extract the specified columns for the given row
    extracted_row1 = df1.loc[row_number, columns_to_extract1]

    # Check if the row has a table
    has_table = extracted_row1["has_table"]

    # Convert the extracted row to a dictionary and remove 'has_table'
    extracted_data1 = extracted_row1.drop("has_table").to_dict()

    # Rename the key to "content"
    extracted_data1["content"] = str(extracted_data1.pop("extracted_content_body"))

    # Convert specific fields to strings and append the desired suffix
    extracted_data1["id"] = str(article_id) + "_content"

    # Wrap the dictionary in a list to match the desired format
    extracted_data_list1 = [extracted_data1]

    # Create unique file name using the article ID
    output_filename1 = f"{article_id}_content.json"

    # Define output path
    output_path1 = os.path.join(output_directory, output_filename1)

    # Export the extracted content body to a JSON file
    with open(output_path1, "w") as json_file:
        json.dump(extracted_data_list1, json_file, indent=4)

    # If there is a table, extract and save it as well
    if has_table:
        extracted_row2 = df1.loc[row_number, columns_to_extract2]
        extracted_data2 = extracted_row2.to_dict()
        extracted_data2["content"] = str(extracted_data2.pop("processed_table_content"))
        extracted_data2["id"] = str(article_id) + "_table"

        # Wrap the dictionary in a list to match the desired format
        extracted_data_list2 = [extracted_data2]

        # Create unique file name for the raw HTML tables
        output_filename2 = f"{article_id}_table.json"
        output_path2 = os.path.join(output_directory, output_filename2)

        # Export the extracted raw HTML tables to a JSON file
        with open(output_path2, "w") as json_file:
            json.dump(extracted_data_list2, json_file, indent=4)

        # Confirm the files were saved
        print(f"JSON files saved at: {output_path1} and {output_path2}")
    else:
        # Confirm the content body file was saved
        print(f"JSON file saved at: {output_path1}")

JSON file saved at: ./data_processing/processed_articles/1442739_content.json
JSON file saved at: ./data_processing/processed_articles/1445332_content.json
JSON file saved at: ./data_processing/processed_articles/1439757_content.json
JSON file saved at: ./data_processing/processed_articles/1439210_content.json
JSON file saved at: ./data_processing/processed_articles/1443217_content.json
JSON file saved at: ./data_processing/processed_articles/1435054_content.json
JSON file saved at: ./data_processing/processed_articles/1437631_content.json
JSON file saved at: ./data_processing/processed_articles/1434620_content.json
JSON file saved at: ./data_processing/processed_articles/1444553_content.json
JSON file saved at: ./data_processing/processed_articles/1445932_content.json
JSON file saved at: ./data_processing/processed_articles/1437772_content.json
JSON file saved at: ./data_processing/processed_articles/1435185_content.json
JSON file saved at: ./data_processing/processed_articles/1435017

#### Remove all .json files from a directory

In [83]:
def remove_json_files(directory):
    """
    Removes all .json files from the specified directory.

    Parameters:
    directory (str): The path to the directory where .json files are to be removed.
    """
    # Create the search pattern for .json files
    search_pattern = os.path.join(directory, "*.json")

    # Get a list of all .json files in the directory
    json_files = glob.glob(search_pattern)

    # Remove each .json file
    for file in json_files:
        os.remove(file)
        print(f"Removed file: {file}")


# Specify the directory where .json files are located
output_directory = "/Users/Richmond/Desktop/syn/HealthierMe2.0/json_test/processed_articles"

# Call the function to remove .json files
remove_json_files(output_directory)